In [ ]:
!pip install transformers torch requests beautifulsoup4 duckduckgo_search

In [ ]:
import torch
from transformers import BertTokenizer, BertForQuestionAnswering
import requests
from bs4 import BeautifulSoup
from duckduckgo_search import ddg

In [ ]:
# Initialize BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [ ]:
def get_definition(word):
    """Retrieves definition from Dictionary.com."""
    url = f"https://www.dictionary.com/browse/{word}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        definition_element = soup.find("span", class_="one-click-content")
        if definition_element:
            return definition_element.get_text()
        else:
            return "Definition not found."
    except requests.exceptions.RequestException as e:
        return f"Error fetching definition: {e}"
    except Exception as e:
        return f"An unexpected error occurred: {e}"

In [ ]:
def get_context_from_ddg(question):
    """Retrieves context from DuckDuckGo search results."""
    try:
        results = ddg(question, max_results=3)  # Get top 3 results
        if results:
            context = " ".join([result["body"] for result in results if result["body"]]) #create one long context string.
            return context
        else:
            return "No context found."
    except Exception as e:
        return f"Error searching DuckDuckGo: {e}"

In [ ]:
def answer_question(question):
    """Answers a question using BERT, DuckDuckGo, and optionally Dictionary.com."""
    context = get_context_from_ddg(question)
    if "No context found." in context or "Error searching DuckDuckGo" in context:
        return context

    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][answer_start:answer_end]))

    answer_tokens = tokenizer.tokenize(answer)
    if len(answer_tokens) == 1 and answer_tokens[0].isalpha():
        definition = get_definition(answer)
        if "Definition not found" not in definition and "Error fetching definition" not in definition and "unexpected error" not in definition:
            answer = f"{answer} : {definition}"

    return answer

In [ ]:
# Example usage
question = "What is the capital of Japan?"
answer = answer_question(question)
print(f"Question: {question}")
print(f"Answer: {answer}")

question2 = "What is photosynthesis?"
answer2 = answer_question(question2)
print(f"Question: {question2}")
print(f"Answer: {answer2}")

question3 = "Define 'serendipity'."
answer3 = answer_question(question3)
print(f"Question: {question3}")
print(f"Answer: {answer3}")